In [11]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.Collecting tensorboard-plugin-wit>=1.6.0

  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [2]:
# Calculate MAPE values
def calculate_mape(actual, predicted):
    return np.mean(np.abs((actual - predicted) / actual)) * 100

In [3]:
# Load the data
data = pd.read_csv('C:\\Users\\tejas\\Desktop\\sample_1 daily.csv', parse_dates=['point_timestamp'], index_col='point_timestamp')

# Set the frequency to daily
data.index = pd.date_range(start=data.index.min(), end=data.index.max(), freq='D')

data = data.drop('Unnamed: 0', axis = 1)


train_data = data.iloc[:int(len(data)*0.8)] #First 80 % is train data

In [4]:
# Arima 

# Fit the model
order = (1, 1, 1) # using an autoregressive term of order 1, a differencing term of order 1, and a moving average term of order 1
model = sm.tsa.arima.ARIMA(train_data['point_value'], order=order)
results = model.fit()

# Generate predictions I am using the last 20% of the data as testing data
start_date = '2021-03-01'
end_date = '2021-07-27'
preds = results.predict(start=start_date, end=end_date, dynamic=False)

# Calculate mean absolute percentage error (MAPE)
actuals = data.loc[start_date:end_date, 'point_value']
mape = calculate_mape(actuals, preds)
print('MAPE:', mape)

MAPE: 4.4752859491734


In [9]:
# SARIMA

train_size = int(len(data) * 0.8)
test_data = data.iloc[int(len(data) * 0.8):]

# Fit the SARIMA model
model = sm.tsa.statespace.SARIMAX(train_data, order=(1, 1, 1), seasonal_order=(0, 0, 0, 7)) # 7 indicates that there is a daily seasonality in the data
results = model.fit()

# Make predictions
predictions = results.forecast(len(test_data))

# Calculate MAPE
mape = calculate_mape(test_data['point_value'], predictions)
print('MAPE:', mape)

MAPE: 4.4752859491734


In [5]:
# XGBoost

test_data = data[int(len(data) * 0.8):]

# Define the features and target variable
features = ['point_value']
target = 'point_value'

# Train the XGBoost model
xgb_model = xgb.XGBRegressor()
xgb_model.fit(train_data[features], train_data[target])

# Make predictions on the test set
xgb_preds = xgb_model.predict(test_data[features])

# Calculate MAPE
mape = calculate_mape(test_data[target], xgb_preds)
print('MAPE:', mape)

MAPE: 0.00035922799334430156


In [6]:
# ETS

test_data = data[int(len(data) * 0.8):]

# Create and fit the ETS model
model = ETSModel(train_data['point_value'].astype(float), error='add', trend='add', seasonal='add', seasonal_periods=7)
result = model.fit()


# Make predictions on the test set
predictions = result.predict(start=test_data.index[0], end=test_data.index[-1])

# Calculate MAPE
mape = calculate_mape(test_data['point_value'], predictions)
print('MAPE:', mape)

MAPE: 5.818020742178061
